In [1]:
# General
import os
import numpy as np
import sys
from glob import glob
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#ML
import tensorflow as tf
from tensorflow.keras import mixed_precision, layers
mixed_precision.set_global_policy("mixed_float16")
AUTO = tf.data.experimental.AUTOTUNE

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
Params = {
    "batch_size": 128
}

In [4]:
def load_dataset(files, window=True):
    dataset = tf.data.TFRecordDataset(files, num_parallel_reads = AUTO)
    
    def _parse_function(example_proto):
        keys_to_feature = {}
        keys_to_feature["TimeSeries"] = tf.io.FixedLenFeature([4096,3], tf.float32)
        keys_to_feature["GroundTruths"] = tf.io.FixedLenFeature([4096,3], tf.float32)
        
        parsed_features = tf.io.parse_single_example(example_proto, keys_to_feature)
        return parsed_features["TimeSeries"], parsed_features["GroundTruths"]
    
    dataset = dataset.map(_parse_function, num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(buffer_size=10000, reshuffle_each_iteration=True)
    dataset = dataset.batch(Params["batch_size"])
#     dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

In [5]:
def get_dataset_files(distances):
    train_files = []
    val_files = []
    for distance in distances:
        data_files = glob(f"../input/synthetic-tfrec/train_{distance}*.tfrec")
        train_files.extend(data_files[:-2])
        val_files.extend(data_files[-2:])
    return train_files, val_files

In [6]:
train_files,val_files = get_dataset_files([100,150,200,250,300])
train_ds = load_dataset(train_files)
val_ds = load_dataset(val_files)
train_files

['../input/synthetic-tfrec\\train_100_12500_00.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_01.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_02.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_03.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_04.tfrec',
 '../input/synthetic-tfrec\\train_100_12500_05.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_00.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_01.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_02.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_03.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_04.tfrec',
 '../input/synthetic-tfrec\\train_150_12500_05.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_00.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_01.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_02.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_03.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_04.tfrec',
 '../input/synthetic-tfrec\\train_200_12500_05.tfrec',
 '../input

In [7]:
def ownInitializer(shape, dtype=None):
    return tf.constant([
        [[1,0,0,0]],
        [[0,1,0,0]],
        [[0,0,1,0]],
        [[0,0,0,1]]
    ],dtype=dtype)

ownInitializer(2,tf.float32)

<tf.Tensor: shape=(4, 1, 4), dtype=float32, numpy=
array([[[1., 0., 0., 0.]],

       [[0., 1., 0., 0.]],

       [[0., 0., 1., 0.]],

       [[0., 0., 0., 1.]]], dtype=float32)>

In [8]:
model = tf.keras.Sequential([
    tf.keras.Input(shape = [4096,3]),
#         layers.Conv1D(kernel_size=12, filters=4,
#                        kernel_initializer=ownInitializer, input_shape=[4096,3], trainable=True, padding="same"),
    layers.Conv1D(kernel_size=4, filters=12, padding="same"),
#     layers.ZeroPadding1D(padding=[0,3]),
    layers.Reshape([4096,12,1]),
    layers.TimeDistributed(layers.Conv1D(kernel_size=3, filters=32, activation="tanh", padding="same")),
#     layers.TimeDistributed(layers.Conv1D(kernel_size=32, filters=64, activation="tanh", padding="same")),
    
    layers.TimeDistributed(layers.MaxPool1D()),
    layers.TimeDistributed(layers.Conv1D(kernel_size=3, filters=16, activation="tanh", padding="same")),
    layers.TimeDistributed(layers.Flatten()),
    
#     layers.Reshape([4093,32]),
    layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128, return_sequences=True)),
    layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128, return_sequences=True)),
    layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128, return_sequences=True)),
    
#     layers.Bidirectional(layers.LSTM(100, return_sequences=True)),
#     layers.Bidirectional(layers.LSTM(100, return_sequences=True)),
#     layers.Bidirectional(layers.LSTM(100, return_sequences=True)),
    
    
#     layers.Bidirectional(layers.LSTM(400)),
    
    
    layers.TimeDistributed(layers.Dense(3, dtype=tf.float32)),
])
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)

model.compile(
    optimizer=opt,
    metrics=["mean_squared_error", "cosine_similarity"],
    loss="mean_squared_error"
)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 4096, 12)          156       
_________________________________________________________________
reshape (Reshape)            (None, 4096, 12, 1)       0         
_________________________________________________________________
time_distributed (TimeDistri (None, 4096, 12, 32)      128       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4096, 6, 32)       0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4096, 6, 16)       1552      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 4096, 96)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 4096, 256)         

In [9]:
train_steps = len(train_files) * 12500 // Params["batch_size"]
val_steps = len(val_files) * 12500 // Params["batch_size"]

In [10]:
from tensorflow.keras.callbacks import *
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.2,
    patience=5, min_lr = 0.000001,
    verbose=1
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta = 0.0003,
    mode="min",
    patience=10
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
#     f"{MDL_PATH}/model_{Params['version']:03}.h5",
    "./model.h5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weight_only=False,
    mode="auto",
    save_freq="epoch"
)


callbacks=[reduce_lr, early_stop, model_checkpoint]

In [ ]:
model.fit(train_ds,validation_data=val_ds, validation_steps=val_steps, steps_per_epoch=train_steps,
          epochs=60, callbacks=callbacks)

Epoch 1/60
6054/6054 [==============================] - 2946s 485ms/step - loss: 0.0116 - mean_squared_error: 0.0116 - cosine_similarity: 0.0404 - val_loss: 0.0100 - val_mean_squared_error: 0.0100 - val_cosine_similarity: 0.0474
Epoch 2/60
6054/6054 [==============================] - 2994s 495ms/step - loss: 0.0090 - mean_squared_error: 0.0090 - cosine_similarity: 0.0626 - val_loss: 0.0075 - val_mean_squared_error: 0.0075 - val_cosine_similarity: 0.0871
Epoch 3/60
6054/6054 [==============================] - 2997s 495ms/step - loss: 0.0083 - mean_squared_error: 0.0083 - cosine_similarity: 0.0727 - val_loss: 0.0070 - val_mean_squared_error: 0.0070 - val_cosine_similarity: 0.0949
Epoch 4/60
6054/6054 [==============================] - 2989s 494ms/step - loss: 0.0069 - mean_squared_error: 0.0069 - cosine_similarity: 0.0969 - val_loss: 0.0065 - val_mean_squared_error: 0.0065 - val_cosine_similarity: 0.1011
Epoch 5/60
6054/6054 [==============================] - 2966s 490ms/step - loss: 0.0

In [ ]:

predictions = model.predict(train_ds, steps=5)

In [ ]:
predictions.shape

In [ ]:
import matplotlib.pyplot as plt
for i in range(1000,1050):
    
    plt.plot(predictions[i])
    plt.show()

In [ ]:
for i in train_ds:
    plt.plot(i[1][7])
    break
    

In [ ]:
model.save("model1.h5")